In [1]:
import streamlit as st
import numpy as np
import pandas as pd
import numpy as np
import requests
import json
import os
import datetime


In [4]:
r = requests.get('https://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']
headers = {'User-agent': useragent}

# Get URL for hourly forecast data

url = f"https://api.weather.gov/points/{37},{-81}"
r = requests.get(url, headers = headers)
print(r)
myjson = json.loads(r.text)
df1 = pd.json_normalize(myjson['properties'])
hourlyURL = df1['forecastHourly'].iloc[0]   

# Obtain actual hourly forecast data

r = requests.get(hourlyURL, headers = headers)

myjson = json.loads(r.text)
df1 = pd.json_normalize(myjson['properties']['periods'])


<Response [200]>


In [5]:
df1

,number,name,startTime,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,windSpeed,windDirection,icon,shortForecast,detailedForecast,probabilityOfPrecipitation.unitCode,probabilityOfPrecipitation.value,dewpoint.unitCode,dewpoint.value,relativeHumidity.unitCode,relativeHumidity.value
0,1,,2024-07-23T18:00:00-04:00,2024-07-23T19:00:00-04:00,False,78,F,,3 mph,S,https://api.weather.gov/icons/land/night/tsra_...,Chance Showers And Thunderstorms,,wmoUnit:percent,50,wmoUnit:degC,20.000000,wmoUnit:percent,71
1,2,,2024-07-23T19:00:00-04:00,2024-07-23T20:00:00-04:00,False,76,F,,2 mph,W,https://api.weather.gov/icons/land/night/tsra_...,Showers And Thunderstorms Likely,,wmoUnit:percent,57,wmoUnit:degC,19.444444,wmoUnit:percent,73
2,3,,2024-07-23T20:00:00-04:00,2024-07-23T21:00:00-04:00,False,73,F,,2 mph,W,https://api.weather.gov/icons/land/night/tsra_...,Patchy Fog,,wmoUnit:percent,60,wmoUnit:degC,19.444444,wmoUnit:percent,81
3,4,,2024-07-23T21:00:00-04:00,2024-07-23T22:00:00-04:00,False,71,F,,2 mph,W,https://api.weather.gov/icons/land/night/tsra_...,Patchy Fog,,wmoUnit:percent,51,wmoUnit:degC,18.888889,wmoUnit:percent,85
4,5,,2024-07-23T22:00:00-04:00,2024-07-23T23:00:00-04:00,False,70,F,,2 mph,W,https://api.weather.gov/icons/land/night/tsra_...,Patchy Fog,,wmoUnit:percent,35,wmoUnit:degC,18.888889,wmoUnit:percent,89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,152,,2024-07-30T01:00:00-04:00,2024-07-30T02:00:00-04:00,False,67,F,,1 mph,SW,https://api.weather.gov/icons/land/night/tsra_...,Slight Chance Showers And Thunderstorms,,wmoUnit:percent,15,wmoUnit:degC,17.222222,wmoUnit:percent,87
152,153,,2024-07-30T02:00:00-04:00,2024-07-30T03:00:00-04:00,False,66,F,,1 mph,SW,https://api.weather.gov/icons/land/night/tsra_...,Slight Chance Showers And Thunderstorms,,wmoUnit:percent,18,wmoUnit:degC,16.666667,wmoUnit:percent,88
153,154,,2024-07-30T03:00:00-04:00,2024-07-30T04:00:00-04:00,False,66,F,,1 mph,SW,https://api.weather.gov/icons/land/night/tsra_...,Slight Chance Showers And Thunderstorms,,wmoUnit:percent,18,wmoUnit:degC,16.666667,wmoUnit:percent,88
154,155,,2024-07-30T04:00:00-04:00,2024-07-30T05:00:00-04:00,False,65,F,,1 mph,SW,https://api.weather.gov/icons/land/night/tsra_...,Slight Chance Showers And Thunderstorms,,wmoUnit:percent,18,wmoUnit:degC,16.666667,wmoUnit:percent,90
